# Validation 和 Cross Validation

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

#### 测试 train_test_split

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [10]:
from sklearn.neighbors import KNeighborsClassifier
best_score, best_p, best_k = 0, 0, 0
for k in range(1, 6):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_score, best_p, best_k = score, p, k
print("best_k: %d" % best_k)
print("best_p: %d" % best_p)
print("best_score: %f" % best_score)

best_k: 5
best_p: 5
best_score: 0.991655


#### 使用交叉验证

In [11]:
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train)

array([ 0.96694215,  0.97493036,  0.98033708])

In [12]:
best_score, best_p, best_k = 0, 0, 0
for k in range(1, 6):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=k, p=p)
        score = cross_val_score(knn_clf, X_test, y_test)
        score = np.mean(score)
        if score > best_score:
            best_score, best_p, best_k = score, p, k
print("best_k: %d" % best_k)
print("best_p: %d" % best_p)
print("best_score: %f" % best_score)

best_k: 1
best_p: 3
best_score: 0.984686


In [14]:
knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=1, p=3)
knn_clf.fit(X_train, y_train)
knn_clf.score(X_test, y_test)

0.9888734353268428

#### 回顾网格搜索

In [15]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {
        'weights': ['distance'],
        'n_neighbors': np.arange(1, 6),
        'p': np.arange(1, 6)
    }
]

In [16]:
knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  1.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['distance'], 'n_neighbors': array([1, 2, 3, 4, 5]), 'p': array([1, 2, 3, 4, 5])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [17]:
grid_search.best_score_

0.98051948051948057

In [18]:
grid_search.best_params_

{'n_neighbors': 4, 'p': 5, 'weights': 'distance'}

In [19]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.99026425591098743

#### cv参数

In [20]:
cross_val_score(knn_clf, X_train, y_train, cv=5)

array([ 0.95909091,  0.97235023,  0.98617512,  0.98122066,  0.98104265])

In [21]:
grid_search = GridSearchCV(knn_clf, param_grid, verbose=1, cv=5)